# Récupération des données sur youtube

Les données utilisées proviennent intégralement de youtube, différentes fonctions sont définies ci-dessous afin de les télécharger et de les analyser



In [ ]:
from __future__ import unicode_literals
import youtube_dl
import ffmpeg
import librosa
import os
from tqdm import tqdm
from pydub.utils import make_chunks
from pydub import AudioSegment
import librosa
import matplotlib.pyplot as plt
import librosa.display
import numpy as np
import seaborn as sns
import gc

In [ ]:
def youtube_download(urls):
    #Téléchargement depuis les urls contenues dans la liste urls
    class MyLogger(object):
        def debug(self, msg):
            pass
        def warning(self, msg):
            pass
        def error(self, msg):
            print(msg)

    def my_hook(d):
        if d['status'] == 'finished':
            print('Done downloading, now converting ...')

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'logger': MyLogger(),
        'progress_hooks': [my_hook],
    }

    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download(urls)
    return True

In [ ]:
def split_files(file_path,directory,name,lenght):
    #séparation d'un fichier audio en fichier de longueur fixe
    
    myaudio = AudioSegment.from_file(file_path , "wav") 
    chunk_length_ms = lenght * 1000 # pydub calculates in millisec
    chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of lenght sec
    
    for i,chunk in enumerate(chunks):
        chunk_name = "{}/{}_{}.wav".format(directory,name,i)
        #print("exporting", chunk_name)
        if i < len(chunk)-2:
            chunk.export(chunk_name, format="wav")
    return True

In [ ]:
def audio_to_jpg(path_in,path_out):
    #conversion d'un fichier audio en jpg (libraire librosa)
    sample,sampling_rate = librosa.load(path_in,sr = 44100)
    stft = librosa.core.stft(sample,n_fft = 4096,hop_length = 512)
    stft_amp = np.abs(stft)
    to_db = librosa.amplitude_to_db(stft_amp)
    fig, ax = plt.subplots(nrows=1, ncols=1) 
    ax = librosa.display.specshow(to_db,sr = sampling_rate,hop_length=512,y_axis = 'log',cmap = 'gray_r')
    fig.savefig(path_out,transparent = True)
    plt.close(fig)
    gc.collect()
    return True

In [ ]:
def creation_arbo(name_project,name_video):
    #création de l'arborescence des données
    if 'dataset' not in os.listdir():
        os.mkdir('dataset')
    
    project_path = 'dataset/{}'.format(name_project)
    if name_project not in os.listdir('dataset'):
        os.mkdir(project_path)
    
    for folder in ['wav','jpg']:
        folder_path = project_path + '/' + folder
        if folder not in os.listdir(project_path):
            os.mkdir(folder_path)
        for folder_2 in name_video:
            if folder_2 not in os.listdir(folder_path):
                os.mkdir(folder_path + '/' + folder_2)
    return True
        
    

In [ ]:
def download_and_converting(name_video,name_project,lenght):
    #Fonction principale: Création arborescence, téléchargement, découpage et conversion des vidéos
    creation_arbo(name_project,name_video)
    for name,video in name_video.items():
        
        found = video.split('=')[1]
        
        test = False
        for i in os.listdir():
            if found in i:
                file_path = i
                test = True
                break
        if not test:
            youtube_download([video])
            for i in os.listdir():
                if found in i:
                    file_path = i
                    test = True
                    break
        
        directory = 'dataset/' + name_project + '/wav/' + name
        split_files(file_path,directory,name,lenght)
        image_directory = 'dataset/' + name_project + '/jpg/' + name
        
        for file in tqdm(os.listdir(directory)):
            path_in = directory + '/'+ file
            jpg_file = file[:-4] + '.jpg'
            path_out = image_directory +'/'+ jpg_file
            if jpg_file not in os.listdir(image_directory):
                audio_to_jpg(path_in,path_out)
    return True

# Download and convert
- Les fonctions ci-dessus permettent de télécharger, depuis un dictionnaire de vidéo, des musiques, les découper en échantillons de longueur *lenght* et de les convertir en jpg

In [ ]:
name_video = {'Chopin':'https://www.youtube.com/watch?v=YzoPmyelSzw',
              'Debussy':'https://www.youtube.com/watch?v=9uKVI6_Fm4g',
              'Beethoven':'https://www.youtube.com/watch?v=0BLaLv-jPQU',
              'Schubert': 'https://www.youtube.com/watch?v=P8ZHToXlp1o'}

name_project  = 'Composer_HD'
lenght = 20 #s

download_and_converting(name_video,name_project,lenght)